In [1]:
from IGraph import *
from AbstractGraph import *
from UGraphModel import *
from DGraphModel import *
#from BayesNet import *

In [2]:
model = DGraphModel()
for idx in range(10):
    model.add(str(idx))

model.connect('0', '1')
model.connect('0', '5')
model.connect('1', '7')
model.connect('3', '2')
model.connect('3', '4')
model.connect('3', '7')
model.connect('3', '8')
model.connect('4', '8')
model.connect('6', '0')
model.connect('6', '1')
model.connect('6', '2')
model.connect('8', '2')
model.connect('8', '7')
model.connect('9', '4')
model.println()

TypeError: add() missing 1 required positional argument: 'tableProb'

In [3]:
DFS = 0
BFS = 1
sorter = TopoSorter(model)
topo = sorter.sort(BFS)
print("topoBFS:", topo)

topo = sorter.sort(DFS)
print("topoDFS:", topo)

topoBFS: []
topoDFS: []


In [4]:
import numpy as np

In [5]:
class ConditionalProbabilityTable():
    def __init__(self,initTable,initParents,initNodeName):
        self.ProbabilityTable = np.array(initTable).T.astype(dtype='<U21')  
        self.ParentsList = initParents
        self.NodeName = initNodeName
        if self.isDiscreteDistribution() == False:
            self.ParentsNum = len(initParents)
            if self.ProbabilityTable.shape[0]-2 != self.ParentsNum:
                raise NameError("Init failed, number of parents is wrong!")
        
    def getValueList(self):
        return list(set(self.ProbabilityTable[-2]))

    def isDiscreteDistribution(self):
        if self.ParentsList == None:
            return True
        else:
            False
    
    def getNodeProbability(self,ParentState):
        RunTable = self.ProbabilityTable
        for name in self.ParentsList:
            if name in ParentState:
                State = np.array(ParentState[name]).astype(dtype='<U21')
                index = np.where( RunTable[0] == State )[0]
                RunTable = RunTable[1:,index]  
            else:
                raise NameError("Parents state is not enough")
        return RunTable
    
    def getOutput(self,ParentState):
        if self.isDiscreteDistribution():   
            sortedProba = self.ProbabilityTable.T[self.ProbabilityTable[1].argsort()].T
            print("sortedProba1: ",sortedProba)
        else:
            NodeProba = self.getNodeProbability(ParentState)
            print("NodeProba: ",NodeProba)
            sortedProba = NodeProba.T[NodeProba[1].argsort()].T     
            print("sortedProba2: ",sortedProba)       
        RandomValue = np.random.random()
        for index in range(len(sortedProba)):
            if RandomValue < sortedProba[1][:index+1].astype(np.float).sum():
                ParentState[self.NodeName] = sortedProba[0,index]
                print("ParentState1: ", ParentState)
                return ParentState
        ParentState[self.NodeName] = sortedProba[0,-1]
        print("ParentState2: ", ParentState)
        return ParentState

        
    def getOutputWithLikehood(self, ParentState, ParentLikelihoodWeight, Observations):
        if self.NodeName not in Observations.keys():
            if self.isDiscreteDistribution():   
                sortedProba = self.ProbabilityTable.T[self.ProbabilityTable[1].argsort()].T
            else:
                NodeProba = self.getNodeProbability(ParentState)
                sortedProba = NodeProba.T[NodeProba[1].argsort()].T
            RandomValue = np.random.random()
            for index in range(len(sortedProba)):
                if RandomValue < sortedProba[1][:index+1].astype(np.float).sum():
                    ParentState[self.NodeName] = sortedProba[0,index]    
                    return ParentState, ParentLikelihoodWeight
            ParentState[self.NodeName] = sortedProba[0,-1]
            return ParentState, ParentLikelihoodWeight
        else:
            if self.isDiscreteDistribution():   
                sortedProba = self.ProbabilityTable.T[self.ProbabilityTable[1].argsort()].T
            else:
                NodeProba = self.getNodeProbability(ParentState)
                sortedProba = NodeProba.T[NodeProba[1].argsort()].T
            index = np.where(sortedProba[0]==np.array(Observations[self.NodeName]).astype(dtype='<U21'))[0][0]
            ParentState[self.NodeName] = sortedProba[0,index]
            ParentLikelihoodWeight[self.NodeName] = float(sortedProba[1,index])
            return ParentState, ParentLikelihoodWeight

# Asia example

In [6]:
asia = ConditionalProbabilityTable( [ ['True', 0.5],[ 'False', 0.5 ]], None, 'asia' )

In [7]:
tuberculosis = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.2 ],
     [ 'True', 'False', 0.8 ],
     [ 'False', 'True', 0.01 ],
     [ 'False', 'False', 0.99 ]], ['asia'],'tuberculosis')

In [8]:
smoking = ConditionalProbabilityTable( [ ['True', 0.5],[ 'False', 0.5 ]], None, 'smoking' )

In [9]:
lung = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.75 ],
     [ 'True', 'False', 0.25 ],
     [ 'False', 'True', 0.02 ],
     [ 'False', 'False', 0.98 ]], ['smoking'], 'lung' )

bronchitis = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.92 ],
     [ 'True', 'False', 0.08 ],
     [ 'False', 'True', 0.03 ],
     [ 'False', 'False', 0.97 ]], ['smoking'], 'bronchitis' )

tuberculosis_or_cancer = ConditionalProbabilityTable(
    [[ 'True', 'True', 'True', 1.0 ],
     [ 'True', 'True', 'False', 0.0 ],
     [ 'True', 'False', 'True', 1.0 ],
     [ 'True', 'False', 'False', 0.0 ],
     [ 'False', 'True', 'True', 1.0 ],
     [ 'False', 'True', 'False', 0.0 ],
     [ 'False', 'False', 'True', 0.0 ],
     [ 'False', 'False', 'False', 1.0 ]], ['tuberculosis', 'lung'], 'tuberculosis_or_cancer' )

In [10]:
xray = ConditionalProbabilityTable(
    [[ 'True', 'True', 0.885 ],
     [ 'True', 'False', 0.115 ],
     [ 'False', 'True', 0.04 ],
     [ 'False', 'False', 0.96 ]], ['tuberculosis_or_cancer'] , 'xray')

In [11]:
dyspnea = ConditionalProbabilityTable(
    [[ 'True', 'True', 'True', 0.96 ],
     [ 'True', 'True', 'False', 0.04 ],
     [ 'True', 'False', 'True', 0.89 ],
     [ 'True', 'False', 'False', 0.11 ],
     [ 'False', 'True', 'True', 0.96 ],
     [ 'False', 'True', 'False', 0.04 ],
     [ 'False', 'False', 'True', 0.89 ],
     [ 'False', 'False', 'False', 0.11 ]], ['tuberculosis_or_cancer', 'bronchitis'],'dyspnea')

In [37]:
NodeList = [asia,smoking,tuberculosis,lung,tuberculosis_or_cancer,bronchitis,xray,dyspnea]
print(type(NodeList[0]))

<class '__main__.ConditionalProbabilityTable'>


# Normal Sampling

In [13]:
RunForwardList = []
for i in range(1):
    RunningVar = {}
    print("asia")
    RunningVar = asia.getOutput(RunningVar)
    print("smoking")
    RunningVar = smoking.getOutput(RunningVar)
    print("tuberculosis")
    RunningVar = tuberculosis.getOutput(RunningVar)
    print("lung")
    RunningVar = lung.getOutput(RunningVar)
    print("tuberculosis_or_cancer")
    RunningVar = tuberculosis_or_cancer.getOutput(RunningVar)
    print("bronchitis")
    RunningVar = bronchitis.getOutput(RunningVar)
    print("xray")
    RunningVar = xray.getOutput(RunningVar)
    print("dyspnea")
    RunningVar = dyspnea.getOutput(RunningVar)
    RunForwardList.append(RunningVar)

asia
sortedProba1:  [['True' 'False']
 ['0.5' '0.5']]
ParentState1:  {'asia': 'False'}
smoking
sortedProba1:  [['True' 'False']
 ['0.5' '0.5']]
ParentState1:  {'asia': 'False', 'smoking': 'False'}
tuberculosis
NodeProba:  [['True' 'False']
 ['0.01' '0.99']]
sortedProba2:  [['True' 'False']
 ['0.01' '0.99']]
ParentState1:  {'asia': 'False', 'smoking': 'False', 'tuberculosis': 'False'}
lung
NodeProba:  [['True' 'False']
 ['0.02' '0.98']]
sortedProba2:  [['True' 'False']
 ['0.02' '0.98']]
ParentState1:  {'asia': 'False', 'smoking': 'False', 'tuberculosis': 'False', 'lung': 'False'}
tuberculosis_or_cancer
NodeProba:  [['True' 'False']
 ['0.0' '1.0']]
sortedProba2:  [['True' 'False']
 ['0.0' '1.0']]
ParentState1:  {'asia': 'False', 'smoking': 'False', 'tuberculosis': 'False', 'lung': 'False', 'tuberculosis_or_cancer': 'False'}
bronchitis
NodeProba:  [['True' 'False']
 ['0.03' '0.97']]
sortedProba2:  [['True' 'False']
 ['0.03' '0.97']]
ParentState1:  {'asia': 'False', 'smoking': 'False', 'tu

In [14]:
Observations = { 'tuberculosis' : 'True', 'smoking' : 'False', 'bronchitis' : 'True' }
print(type(Observations))

<class 'dict'>


In [15]:
NodeNameList = ['asia','smoking','tuberculosis','lung','tuberculosis_or_cancer','bronchitis','xray','dyspnea']

In [16]:
RunValueList = []
KeyList = RunForwardList[0].keys()
for RunForwardValue in RunForwardList:
    OneRow = []
    ObservationsCheck = True
    for key in Observations:
        if Observations[key] != RunForwardValue[key]:
            ObservationsCheck = False
    if ObservationsCheck == True:
        for key in RunForwardValue: 
            OneRow.append(RunForwardValue[key])
        RunValueList.append(OneRow)
NumpyRunValue = np.array(RunValueList)
NumpyKey = np.array(list(KeyList))

In [17]:
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,len(np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0])/NumpyRunValue.shape[0])
    print('============================')

asia


IndexError: too many indices for array

# Likelihood

In [18]:
RunForwardList = []
RunForwardWeight = []
for i in range(100000):
    w = 1
    RunningVar = {}
    ParentLikelihoodWeight = {}
    Observations = { 'tuberculosis' : 'True', 'smoking' : 'False', 'bronchitis' : 'True' }
    RunningVar,ParentLikelihoodWeight = asia.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = smoking.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = tuberculosis.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = lung.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = tuberculosis_or_cancer.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = bronchitis.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = xray.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = dyspnea.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunForwardList.append(RunningVar)
    for key in Observations:
        w *= ParentLikelihoodWeight[key]
    RunForwardWeight.append(w)

NameError: name 'random' is not defined

In [19]:
RunValueList = []
WeightList = []
KeyList = []
for index_,RunForwardValue in enumerate(RunForwardList):
    OneRow = []
    for key in RunForwardValue: 
        if key not in Observations:
            if index_ == 0:
                KeyList.append(key)
            OneRow.append(RunForwardValue[key])
    RunValueList.append(OneRow)
    WeightList.append(RunForwardWeight[index_])
NumpyRunValue = np.array(RunValueList)
NumpyKey = np.array(list(KeyList))
NumpyWeight = np.array(WeightList)

In [20]:
TotalWeight = np.sum(NumpyWeight)
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,np.sum(NumpyWeight[np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0]]) / TotalWeight)
    print('============================')

asia


IndexError: index 0 is out of bounds for axis 0 with size 0

# Monty Hall  Example 

In [21]:
guest = ConditionalProbabilityTable( [ ['A', 1./3],[ 'B', 1./3 ],[ 'C', 1./3 ]], None, 'guest' )
prize = ConditionalProbabilityTable( [ ['A', 1./3],[ 'B', 1./3 ],[ 'C', 1./3 ]], None, 'prize' )

In [22]:
monty = ConditionalProbabilityTable( 
    [[ 'A', 'A', 'A', 0.0 ],
     [ 'A', 'A', 'B', 0.5 ],
     [ 'A', 'A', 'C', 0.5 ],
     [ 'A', 'B', 'A', 0.0 ],
     [ 'A', 'B', 'B', 0.0 ],
     [ 'A', 'B', 'C', 1.0 ],
     [ 'A', 'C', 'A', 0.0 ],
     [ 'A', 'C', 'B', 1.0 ],
     [ 'A', 'C', 'C', 0.0 ],
     [ 'B', 'A', 'A', 0.0 ],
     [ 'B', 'A', 'B', 0.0 ],
     [ 'B', 'A', 'C', 1.0 ],
     [ 'B', 'B', 'A', 0.5 ],
     [ 'B', 'B', 'B', 0.0 ],
     [ 'B', 'B', 'C', 0.5 ],
     [ 'B', 'C', 'A', 1.0 ],
     [ 'B', 'C', 'B', 0.0 ],
     [ 'B', 'C', 'C', 0.0 ],
     [ 'C', 'A', 'A', 0.0 ],
     [ 'C', 'A', 'B', 1.0 ],
     [ 'C', 'A', 'C', 0.0 ],
     [ 'C', 'B', 'A', 1.0 ],
     [ 'C', 'B', 'B', 0.0 ],
     [ 'C', 'B', 'C', 0.0 ],
     [ 'C', 'C', 'A', 0.5 ],
     [ 'C', 'C', 'B', 0.5 ],
     [ 'C', 'C', 'C', 0.0 ]], ['guest','prize'], 'monty' )

In [23]:
Observations = { 'guest' : 'A' }

In [24]:
NodeList = [guest,prize,monty]

In [25]:
RunForwardList = []
RunForwardWeight = []
for i in range(100000):
    w = 1
    RunningVar = {}
    ParentLikelihoodWeight = {}
    RunningVar,ParentLikelihoodWeight = guest.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = prize.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunningVar,ParentLikelihoodWeight = monty.getOutputWithLikehood(RunningVar,ParentLikelihoodWeight,Observations)
    RunForwardList.append(RunningVar)
    for key in Observations:
        w *= ParentLikelihoodWeight[key]
    RunForwardWeight.append(w)

NameError: name 'random' is not defined

In [26]:
RunValueList = []
WeightList = []
KeyList = []
for index_,RunForwardValue in enumerate(RunForwardList):
    OneRow = []
    for key in RunForwardValue: 
        if key not in Observations:
            if index_ == 0:
                KeyList.append(key)
            OneRow.append(RunForwardValue[key])
    RunValueList.append(OneRow)
    WeightList.append(RunForwardWeight[index_])
NumpyRunValue = np.array(RunValueList)
NumpyKey = np.array(list(KeyList))
NumpyWeight = np.array(WeightList)

In [27]:
TotalWeight = np.sum(NumpyWeight)
for Node in NodeList:
    if Node.NodeName in Observations.keys():
        print(Node.NodeName, Observations[Node.NodeName])
    else:
        print(Node.NodeName)
        indexNumpyRunValue = np.where(NumpyKey==Node.NodeName)[0][0]
        for Value in Node.getValueList():
            print(Value,np.sum(NumpyWeight[np.where(NumpyRunValue[:,indexNumpyRunValue] == Value)[0]]) / TotalWeight)
    print('============================')

guest A
prize


IndexError: index 0 is out of bounds for axis 0 with size 0

In [35]:
a = ['i']
b = ['']
print(b[0])
if not a:
    print("List a is empty")
else:
    print("List a is not empty")

if b == ['']:
    print("List b is empty")
else:
    print("List b is not empty")



List a is not empty
List b is empty


In [6]:
file = open("test.txt")
for line in file:
    ObservationList = dict()
    Probability = dict()
    print("line: ",line)
    if len(line) == 2:
        numVertex = line.split('\n')[0]
    else:
        inputGraph = line.split('\n')[0]
        input = inputGraph.split(';')
        print("input: ", input)
        prob = str(input[0])
        print("prob: ", prob)
        for subProb in prob.split(','):
            print("subProb: ",subProb)
            subP = subProb.split('=')
            print("subP: ", subP)
            Probability.update({subP[0]:subP[1]})
        condition = input[1].split(',')
        print("condition: ", condition)
        for subcondition in condition:
            if subcondition == ' ':
                ObservationList = []
                break
            sub = subcondition.split('=')
            print("subcondition: ",sub)
            ObservationList.update({sub[0]:sub[1]})
        print("Probability: ",Probability)
        print("ObservationList: ",ObservationList)
        print("=======")


line:  3

line:  G=A;I=Cao,D=Kho

input:  ['G=A', 'I=Cao,D=Kho']
prob:  G=A
subProb:  G=A
subP:  ['G', 'A']
condition:  ['I=Cao', 'D=Kho']
subcondition:  ['I', 'Cao']
subcondition:  ['D', 'Kho']
Probability:  {'G': 'A'}
ObservationList:  {'I': 'Cao', 'D': 'Kho'}
line:  G=A,L=Cao;I=Cao,D=Kho

input:  ['G=A,L=Cao', 'I=Cao,D=Kho']
prob:  G=A,L=Cao
subProb:  G=A
subP:  ['G', 'A']
subProb:  L=Cao
subP:  ['L', 'Cao']
condition:  ['I=Cao', 'D=Kho']
subcondition:  ['I', 'Cao']
subcondition:  ['D', 'Kho']
Probability:  {'G': 'A', 'L': 'Cao'}
ObservationList:  {'I': 'Cao', 'D': 'Kho'}
line:  G=B,S=Cao; 

input:  ['G=B,S=Cao', ' ']
prob:  G=B,S=Cao
subProb:  G=B
subP:  ['G', 'B']
subProb:  S=Cao
subP:  ['S', 'Cao']
condition:  [' ']
Probability:  {'G': 'B', 'S': 'Cao'}
ObservationList:  []


In [21]:
file = open("test.txt")
lines = file.readlines()
print(lines)
for i in range(len(lines)):
    lines[i] = lines[i].strip()

print(lines)
while lines.count('') > 0:
    lines.remove('')

print(lines)

['2 \n', '\n', ' \n', 'G=A;I=Cao,D=Kho  \n', 'G=A,L=Manh;I=Cao,D=Kho    \n', 'G=B,S=Cao;\n', '\n', '  ']
['2', '', '', 'G=A;I=Cao,D=Kho', 'G=A,L=Manh;I=Cao,D=Kho', 'G=B,S=Cao;', '', '']
['2', 'G=A;I=Cao,D=Kho', 'G=A,L=Manh;I=Cao,D=Kho', 'G=B,S=Cao;']


In [6]:
print(string_without_empty_lines)

2

G=A;I=Cao,D=Kho

G=A,L=Manh;I=Cao,D=Kho

G=B,S=Cao; 




In [180]:
import numpy as np

def uniqueElement(observation):
    # intilize a null list
    unique_list = []
    # traverse for all elements
    for x in observation:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

def get_value(factor, key):
    newd = dict()
    for i in range(len(factor)):
        newd.update({tuple([factor[i][j] for j in range(len(factor[i])-1)]):factor[i][-1]})
    return newd.get(key)

def swapCommon(factor,key,begin):
    if type(factor[0][-1]) is float:
        if begin: #swap common to the first row
            print(len(factor))
            for row in range(len(factor)):
                tmp = factor[row][0]
                factor[row][0] = factor[row][key]
                factor[row][key] = tmp
        else: # swap common to the latest row
            print(len(factor))
            for row in range(len(factor)):
                tmp = factor[row][-2]
                factor[row][-2] = factor[row][key]
                factor[row][key] = tmp
    else:
        if begin: #swap common to the first row
            print(len(factor))
            tmp = factor[0]
            factor[0] = factor[key]
            factor[key] = tmp
        else: # swap common to the latest row
            print(len(factor))
            tmp = factor[-1]
            factor[-1] = factor[key]
            factor[key] = tmp

def parsing_combinationProb(listProb):
        #print("tableValue: ", tableValue)
        output = []
        n = len(listProb)
        total = 1
        for i in range(n):
            total *= len(listProb[i])
        index = np.zeros(n)
        index[-1] = -1

        count = 0
        while count < total:
            if index[n-1] != len(listProb[n-1]) - 1:
                index[n-1] += 1
            else:
                index[n-1] = 0
                preceding = n - 1
                while True:
                    preceding -= 1
                    if index[preceding] != len(listProb[preceding]) - 1:
                        index[preceding] += 1
                        break
                    else:
                        index[preceding] = 0
            aResult = []
            for i in range(n):
                aResult.append(listProb[i][int(index[i])])
            #print("count: ", count)
            output.append(aResult)
            count += 1
        #print(output)
        return output


In [181]:
xray=[[ 'True', 'True', 0.885 ],
     [ 'True', 'False', 0.115 ],
     [ 'False', 'True', 0.04 ],
     [ 'False', 'False', 0.96 ]]
xray

[['True', 'True', 0.885],
 ['True', 'False', 0.115],
 ['False', 'True', 0.04],
 ['False', 'False', 0.96]]

In [182]:
tt = ('True','False')
print(get_value(xray,tt))

0.115


In [183]:
kaka = [[ 'Small', 'Guitar', 'A', 0.0 ],
        [ 'Small', 'Guitar', 'B', 0.5 ],
        [ 'Small', 'Guitar', 'C', 0.5 ],
        [ 'Small', 'Bass', 'A', 0.0 ],
        [ 'Small', 'Bass', 'B', 0.0 ],
        [ 'Small', 'Bass', 'C', 1.0 ],
        [ 'Small', 'Drum', 'A', 0.0 ],
        [ 'Small', 'Drum', 'B', 1.0 ],
        [ 'Small', 'Drum', 'C', 0.0 ],
        [ 'Medium', 'Guitar', 'A', 0.0 ],
        [ 'Medium', 'Guitar', 'B', 0.0 ],
        [ 'Medium', 'Guitar', 'C', 1.0 ],
        [ 'Medium', 'Bass', 'A', 0.5 ],
        [ 'Medium', 'Bass', 'B', 0.0 ],
        [ 'Medium', 'Bass', 'C', 0.5 ],
        [ 'Medium', 'Drum', 'A', 1.0 ],
        [ 'Medium', 'Drum', 'B', 0.0 ],
        [ 'Medium', 'Drum', 'C', 0.0 ],
        [ 'Big', 'Guitar', 'A', 0.0 ],
        [ 'Big', 'Guitar', 'B', 1.0 ],
        [ 'Big', 'Guitar', 'C', 0.0 ],
        [ 'Big', 'Bass', 'A', 1.0 ],
        [ 'Big', 'Bass', 'B', 0.0 ],
        [ 'Big', 'Bass', 'C', 0.0 ],
        [ 'Big', 'Drum', 'A', 0.5 ],
        [ 'Big', 'Drum', 'B', 0.5 ],
        [ 'Big', 'Drum', 'C', 0.0 ]]
order_kaka = ['I', 'S', 'G']

xray=[[ 'Low', 'A', 0.885 ],
     [ 'Low', 'B', 0.115 ],
     [ 'Low', 'C', 0.04 ],
     [ 'High', 'A', 0.96 ],
     [ 'High', 'B', 0.96 ],
     [ 'High', 'C', 0.96 ]]
order_xray = ['K', 'G']

In [184]:
f1_vars = order_kaka
f2_vars = order_xray
# first, find all the common variables between the two factors:
common_vars = []
f1 = []
f2 = []
for char in f1_vars:
    if char in f2_vars:
        common_vars.append(char)
    else:
        f1.append(char)

for char in f2_vars:
    if char not in f1_vars:
        f2.append(char)

all_vars = f1+common_vars+f2
num_vars = len(all_vars)
print("comon_var: ", common_vars)
print("f1: ", f1)
print("f2: ", f2)
print("all_vars: ", all_vars)
print("num_vars: ", num_vars)

comon_var:  ['G']
f1:  ['I', 'S']
f2:  ['K']
all_vars:  ['I', 'S', 'G', 'K']
num_vars:  4


In [201]:
#swap common var in original table
f1_str = all_vars[:len(f1_vars)]
f2_str = all_vars[len(f2_vars):]
print(f1_str)
print(f2_str[1:])
print(len(f1_vars))
print(len(f2_vars))

['I', 'S', 'G']
['K']
3
2


In [186]:
index_common_f1 = order_kaka.index(common_vars[0])
index_common_f2 = order_xray.index(common_vars[0])
print(index_common_f1)
print(index_common_f2)

2
1


In [187]:
swapCommon(kaka,index_common_f1,False)
swapCommon(order_kaka,index_common_f1,False)
swapCommon(xray,index_common_f2,True)
swapCommon(order_xray,index_common_f2,True)

27
3
6
2


In [188]:
print(order_xray)

['G', 'K']


In [189]:
print(xray)

[['A', 'Low', 0.885], ['B', 'Low', 0.115], ['C', 'Low', 0.04], ['A', 'High', 0.96], ['B', 'High', 0.96], ['C', 'High', 0.96]]


In [190]:
listI = uniqueElement([kaka[i][order_kaka.index('I')] for i in range(len(kaka))])
listS = uniqueElement([kaka[i][order_kaka.index('S')] for i in range(len(kaka))])
listG = uniqueElement([kaka[i][order_kaka.index('G')] for i in range(len(kaka))])
listK = uniqueElement([xray[i][order_xray.index('K')] for i in range(len(xray))])

In [191]:
print("listI:",listI)
print("listS:",listS)
print("listG:",listG)
print("listK:",listK)

listI: ['Small', 'Medium', 'Big']
listS: ['Guitar', 'Bass', 'Drum']
listG: ['A', 'B', 'C']
listK: ['Low', 'High']


In [192]:
listmerge = []
listmerge.append(listI)
listmerge.append(listS)
listmerge.append(listG)
listmerge.append(listK)
listmerge

[['Small', 'Medium', 'Big'],
 ['Guitar', 'Bass', 'Drum'],
 ['A', 'B', 'C'],
 ['Low', 'High']]

In [199]:
result = parsing_combinationProb(listmerge)
result

[['Small', 'Guitar', 'A', 'Low'],
 ['Small', 'Guitar', 'A', 'High'],
 ['Small', 'Guitar', 'B', 'Low'],
 ['Small', 'Guitar', 'B', 'High'],
 ['Small', 'Guitar', 'C', 'Low'],
 ['Small', 'Guitar', 'C', 'High'],
 ['Small', 'Bass', 'A', 'Low'],
 ['Small', 'Bass', 'A', 'High'],
 ['Small', 'Bass', 'B', 'Low'],
 ['Small', 'Bass', 'B', 'High'],
 ['Small', 'Bass', 'C', 'Low'],
 ['Small', 'Bass', 'C', 'High'],
 ['Small', 'Drum', 'A', 'Low'],
 ['Small', 'Drum', 'A', 'High'],
 ['Small', 'Drum', 'B', 'Low'],
 ['Small', 'Drum', 'B', 'High'],
 ['Small', 'Drum', 'C', 'Low'],
 ['Small', 'Drum', 'C', 'High'],
 ['Medium', 'Guitar', 'A', 'Low'],
 ['Medium', 'Guitar', 'A', 'High'],
 ['Medium', 'Guitar', 'B', 'Low'],
 ['Medium', 'Guitar', 'B', 'High'],
 ['Medium', 'Guitar', 'C', 'Low'],
 ['Medium', 'Guitar', 'C', 'High'],
 ['Medium', 'Bass', 'A', 'Low'],
 ['Medium', 'Bass', 'A', 'High'],
 ['Medium', 'Bass', 'B', 'Low'],
 ['Medium', 'Bass', 'B', 'High'],
 ['Medium', 'Bass', 'C', 'Low'],
 ['Medium', 'Bass', 'C'

In [200]:
for row in range(len(result)):
    key1 = result[row][:len(f1_vars)]
    key2 = result[row][len(f2_vars):]
    #print("key1: ",key1)
    #print("key2: ",key2)
    value1 = get_value(kaka,tuple(key1))
    value2 = get_value(xray,tuple(key2))
    #print("value1: ",value1)
    #print("value2: ",value2)
    product = value1 * value2
    result[row].append(product)
result

[['Small', 'Guitar', 'A', 'Low', 0.0],
 ['Small', 'Guitar', 'A', 'High', 0.0],
 ['Small', 'Guitar', 'B', 'Low', 0.0575],
 ['Small', 'Guitar', 'B', 'High', 0.48],
 ['Small', 'Guitar', 'C', 'Low', 0.02],
 ['Small', 'Guitar', 'C', 'High', 0.48],
 ['Small', 'Bass', 'A', 'Low', 0.0],
 ['Small', 'Bass', 'A', 'High', 0.0],
 ['Small', 'Bass', 'B', 'Low', 0.0],
 ['Small', 'Bass', 'B', 'High', 0.0],
 ['Small', 'Bass', 'C', 'Low', 0.04],
 ['Small', 'Bass', 'C', 'High', 0.96],
 ['Small', 'Drum', 'A', 'Low', 0.0],
 ['Small', 'Drum', 'A', 'High', 0.0],
 ['Small', 'Drum', 'B', 'Low', 0.115],
 ['Small', 'Drum', 'B', 'High', 0.96],
 ['Small', 'Drum', 'C', 'Low', 0.0],
 ['Small', 'Drum', 'C', 'High', 0.0],
 ['Medium', 'Guitar', 'A', 'Low', 0.0],
 ['Medium', 'Guitar', 'A', 'High', 0.0],
 ['Medium', 'Guitar', 'B', 'Low', 0.0],
 ['Medium', 'Guitar', 'B', 'High', 0.0],
 ['Medium', 'Guitar', 'C', 'Low', 0.04],
 ['Medium', 'Guitar', 'C', 'High', 0.96],
 ['Medium', 'Bass', 'A', 'Low', 0.4425],
 ['Medium', 'Bas

In [1]:
from FactorUtilities import *

ft = FactorUtilities()

kaka = [[ 'Small', 'Guitar', 'A', 0.0 ],
        [ 'Small', 'Guitar', 'B', 0.5 ],
        [ 'Small', 'Guitar', 'C', 0.5 ],
        [ 'Small', 'Bass', 'A', 0.0 ],
        [ 'Small', 'Bass', 'B', 0.0 ],
        [ 'Small', 'Bass', 'C', 1.0 ],
        [ 'Small', 'Drum', 'A', 0.0 ],
        [ 'Small', 'Drum', 'B', 1.0 ],
        [ 'Small', 'Drum', 'C', 0.0 ],
        [ 'Medium', 'Guitar', 'A', 0.0 ],
        [ 'Medium', 'Guitar', 'B', 0.0 ],
        [ 'Medium', 'Guitar', 'C', 1.0 ],
        [ 'Medium', 'Bass', 'A', 0.5 ],
        [ 'Medium', 'Bass', 'B', 0.0 ],
        [ 'Medium', 'Bass', 'C', 0.5 ],
        [ 'Medium', 'Drum', 'A', 1.0 ],
        [ 'Medium', 'Drum', 'B', 0.0 ],
        [ 'Medium', 'Drum', 'C', 0.0 ],
        [ 'Big', 'Guitar', 'A', 0.0 ],
        [ 'Big', 'Guitar', 'B', 1.0 ],
        [ 'Big', 'Guitar', 'C', 0.0 ],
        [ 'Big', 'Bass', 'A', 1.0 ],
        [ 'Big', 'Bass', 'B', 0.0 ],
        [ 'Big', 'Bass', 'C', 0.0 ],
        [ 'Big', 'Drum', 'A', 0.5 ],
        [ 'Big', 'Drum', 'B', 0.5 ],
        [ 'Big', 'Drum', 'C', 0.0 ]]
order_kaka = ['I', 'S', 'G']

xray=[[ 'Low', 'A', 0.885 ],
     [ 'Low', 'B', 0.115 ],
     [ 'Low', 'C', 0.04 ],
     [ 'High', 'A', 0.96 ],
     [ 'High', 'B', 0.96 ],
     [ 'High', 'C', 0.96 ]]
order_xray = ['K', 'G']

In [2]:
rl = ft.multiply(kaka,order_kaka,xray,order_xray)
rl

[['Small', 'Guitar', 'A', 'Low', 0.0],
 ['Small', 'Guitar', 'A', 'High', 0.0],
 ['Small', 'Guitar', 'B', 'Low', 0.0575],
 ['Small', 'Guitar', 'B', 'High', 0.48],
 ['Small', 'Guitar', 'C', 'Low', 0.02],
 ['Small', 'Guitar', 'C', 'High', 0.48],
 ['Small', 'Bass', 'A', 'Low', 0.0],
 ['Small', 'Bass', 'A', 'High', 0.0],
 ['Small', 'Bass', 'B', 'Low', 0.0],
 ['Small', 'Bass', 'B', 'High', 0.0],
 ['Small', 'Bass', 'C', 'Low', 0.04],
 ['Small', 'Bass', 'C', 'High', 0.96],
 ['Small', 'Drum', 'A', 'Low', 0.0],
 ['Small', 'Drum', 'A', 'High', 0.0],
 ['Small', 'Drum', 'B', 'Low', 0.115],
 ['Small', 'Drum', 'B', 'High', 0.96],
 ['Small', 'Drum', 'C', 'Low', 0.0],
 ['Small', 'Drum', 'C', 'High', 0.0],
 ['Medium', 'Guitar', 'A', 'Low', 0.0],
 ['Medium', 'Guitar', 'A', 'High', 0.0],
 ['Medium', 'Guitar', 'B', 'Low', 0.0],
 ['Medium', 'Guitar', 'B', 'High', 0.0],
 ['Medium', 'Guitar', 'C', 'Low', 0.04],
 ['Medium', 'Guitar', 'C', 'High', 0.96],
 ['Medium', 'Bass', 'A', 'Low', 0.4425],
 ['Medium', 'Bas